In [1]:
using Interact
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using Ipopt

In [12]:
import Flash
reload("Flash")

In [13]:
squished = Flash.DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [14]:
Flash.DepthData.render_lcmgl(squished)

In [15]:
unsquished = Flash.DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [16]:
Flash.DepthData.render_lcmgl(unsquished)

In [17]:
model = Flash.Models.squishable()
state = Flash.ManipulatorState(model)
Flash.draw(state)

DrakeVisualizer.Visualizer{Int64}(DataStructures.OrderedDict(1=>DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 294xFixedSizeArrays.Point{3,Float64},     faces: 584xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))]),1,PyLCM.LCM(PyObject <LCM object at 0x316860db0>))

In [18]:
point_cloud = unsquished
Flash.DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points];

In [19]:
function test_gradient_descent(model, sensed_points, loops, solver)
    for i = 1:loops
        
        function callback{T}(x::AbstractVector{T}, c)
            x_value = Flash.value.(x)
            state = Flash.ManipulatorState(model)
            Flash.GradientDescent.unflatten!(state, x_value)
            Flash.draw(state, false)
        end
        
        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        xstar = Flash.Tracking.estimate_state(model, sensed_points, x_estimated; callback=callback, solver=solver)
        state = Flash.ManipulatorState(model)
        Flash.GradientDescent.unflatten!(state, xstar)
        Flash.draw(state, true)
    end
end


test_gradient_descent (generic function with 1 method)

In [20]:
num_vars = Flash.num_states(model)
solver = NaiveSolver(num_vars; rate=0.01, max_step=0.1, iteration_limit=50, 
        precondition_divisors=vcat([1 for i in 1:7], [1e-3 for i in 8:num_vars]))
test_gradient_descent(model, sensed_points, 1, solver);

In [21]:
@time test_gradient_descent(model, sensed_points, 1, solver);

  8.083380 seconds (45.00 M allocations: 3.752 GB, 7.66% gc time)


In [22]:
ipopt_solver = IpoptSolver(print_level=1, max_iter=50)
test_gradient_descent(model, sensed_points, 1, ipopt_solver);